In [ ]:
import pandas as pd
import pickle
import requests as rq
import json

In [ ]:
df = pd.read_pickle("data.pkl")

In [48]:
with open('isocodes.pkl', 'rb') as handle:
  isocodes = pickle.load(handle)
isocodes

{'Afghanistan': {'iso3': 'AFG', 'iso2': 'AF'},
 'Albania': {'iso3': 'ALB', 'iso2': 'AL'},
 'Armenia': {'iso3': 'ARM', 'iso2': 'AM'},
 'Azerbaijan': {'iso3': 'AZE', 'iso2': 'AZ'},
 'Bahrain': {'iso3': 'BHR', 'iso2': 'BH'},
 'Bangladesh': {'iso3': 'BGD', 'iso2': 'BD'},
 'Belarus': {'iso3': 'BLR', 'iso2': 'BY'},
 'Bhutan': {'iso3': 'BTN', 'iso2': 'BT'},
 'Bosnia and Herzegovina': {'iso3': 'BIH', 'iso2': 'BA'},
 'Brunei': {'iso3': 'BRN', 'iso2': 'BN'},
 'Bulgaria': {'iso3': 'BGR', 'iso2': 'BG'},
 'Cambodia': {'iso3': 'KHM', 'iso2': 'KH'},
 'China': {'iso3': 'CHN', 'iso2': 'CN'},
 'Croatia': {'iso3': 'HRV', 'iso2': 'HR'},
 'Czech Republic': {'iso3': 'CZE', 'iso2': 'CZ'},
 'East Timor': {'iso3': 'TLS', 'iso2': 'TL'},
 'Egypt': {'iso3': 'EGY', 'iso2': 'EG'},
 'Estonia': {'iso3': 'EST', 'iso2': 'EE'},
 'France (Not BRI)': {'iso3': 'FRA', 'iso2': 'FR'},
 'Georgia': {'iso3': 'GEO', 'iso2': 'GE'},
 'Germany (not BRI)': {'iso3': 'DEU', 'iso2': 'DE'},
 'Hungary': {'iso3': 'HUN', 'iso2': 'HU'},
 'In

In [ ]:
with open('apidata.json', 'r') as handle:
  data = json.load(handle)

In [ ]:
yidaiyilu_data = {}

In [ ]:
# Request part
# Need to add a proxy to get the request - template of request
'''
apiURL = "https://eng.yidaiyilu.gov.cn/wcm.files/data/data-query-20170525.effix"
headers = {
    'Accept': "application/json, text/javascript, */*; q=0.01",
    'Referer': "https://eng.yidaiyilu.gov.cn/dsjym.htm",
    'X-Requested-With': "XMLHttpRequest",
    'Sec-Fetch-Mode': "cors",
    'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36",
    'X-Request-Id': "1150;r=19560479",
    'cache-control': "no-cache",
    'Cookie' : "__jsluid_s=a0854fc5fcc55ebf2f523fc55744cae0; path=/; domain=.eng.yidaiyilu.gov.cn; Secure; HttpOnly; Expires=Tue, 13 Oct 2020 02:15:44 GMT;"
}

resp = rq.get(apiURL, headers)
        
# Error handler
if(resp.status_code != 200) :
    print("STATUS_CODE ERROR", resp.status_code, resp.text)
else:
    # Response handler
    data = resp.json()
    data["country_macro"]
'''

In [53]:
# 2015 year
year = "65d2ea03425887a717c435081cfc5dbb" 

indicators = {
    "9b688b5fbb441714ba73f4379e7b135d": "BRI GDP",
    "5c4591fe4746333f6e7b66b13ad980a2": "BRI GDP growth rate",
    "6d602224213550f3ee07477f1654f326": "BRI GDP p.c.",
    "07f451a8c9b2006fb937d1eb268ac60b": "BRI Y.o.Y. GDP growth rate p.c.",
    "c1329b4c01d4c08a68b8a280fb5a90ec": "BRI Agriculture value added per sector ",
    "f00273fa66d45147b2dd4dadd6275d32": "BRI Industry value added per sector ",
    "41fedd97a26d4782e531b5b4c67622b7": "BRI Services value added per sector ",
    "7b8d0a597b57bab793f428193eb899fe": "BRI Internet penetration rate (%)",
    "2a048dbe6ff2ee51d07a6314752c87d7": "BRI Population density (/km2)",
    "85bf6dab128029be54a3b1e643f0aa71": "BRI Percentage of Urban Population in Total Population"
}

for key_country in data['country_macro']['term']['area']:
    full_country_name = data['country_macro']['term']['area'][key_country]['full']
    short_country_name = data['country_macro']['term']['area'][key_country]['short']
    
    # Check if we care about this country
    if full_country_name not in isocodes and short_country_name not in isocodes:
        continue
    else:
        yidaiyilu_data[full_country_name] = {}
    
    for key_indic in data['country_macro']['data'][year][key_country]:
        # Check if we care about this indicator
        if key_indic not in indicators:
            continue
        value = data['country_macro']['data'][year][key_country][key_indic]
        if value == '':
            value = 0
        yidaiyilu_data[full_country_name].update({indicators[key_indic] : value})

In [51]:
len(yidaiyilu_data)

In [55]:
yidaiyilu_data

{'Albania': {'BRI GDP': '11455595709.1413',
  'BRI GDP growth rate': '2.55999999999999',
  'BRI GDP p.c.': '3965.01680558488',
  'BRI Y.o.Y. GDP growth rate p.c.': '2.71928006930717',
  'BRI Agriculture value added per sector ': '22.0492060503358',
  'BRI Industry value added per sector ': '26.1513054666415',
  'BRI Services value added per sector ': '51.7994884830226',
  'BRI Population density (/km2)': '105.444051094891',
  'BRI Percentage of Urban Population in Total Population': '57.407'},
 'Afghanistan': {'BRI GDP': '19199437988.8023',
  'BRI GDP growth rate': '1.51991126448578',
  'BRI GDP p.c.': '590.269515382605',
  'BRI Y.o.Y. GDP growth rate p.c.': '-1.28616720583651',
  'BRI Agriculture value added per sector ': '22.6039268726035',
  'BRI Industry value added per sector ': '22.4166817955447',
  'BRI Services value added per sector ': '54.9793913318518',
  'BRI Population density (/km2)': '49.8216493582085',
  'BRI Percentage of Urban Population in Total Population': '26.703'

In [ ]:
# Add data to df
for country in yidaiyilu_data.keys():
    for indic in yidaiyilu_data[country]:
        if pd.isnull( df[indic].iloc[list(yidaiyilu_data).index(country)]):
            df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]